# Scraping Google Classroom Part 2
**This file is made by Ali Nurdin. You can use this to scrap course in Google Classroom as an instructor.**

In [1]:
# Importing Selenium and Time
from selenium import webdriver
import time

In [2]:
# Opening Web Driver and get to url
Driver = webdriver.Chrome()
Driver.get("https://classroom.google.com/u/0/h")
time.sleep(10)

In [3]:
# Asking user to enter their email and password. This will be used for login to google classroom automatically
Email = input("Please input your email: ").strip()
Password = input("Please input your password").strip()

In [4]:
# Logging automatically using data that has been stored
Driver.find_element_by_id("identifierId").send_keys(Email)
Driver.find_element_by_css_selector("div.VfPpkd-RLmnJb").click()
time.sleep(5)
Driver.find_element_by_css_selector("input.whsOnd.zHQkBf").send_keys(Password)
Driver.find_element_by_css_selector("div.VfPpkd-RLmnJb").click()
time.sleep(3)

In [5]:
#Searching Class
def search_class():
    while True:
        i = 0
        Classes = Driver.find_elements_by_css_selector("div.Tc9hUd")
        for Class in Classes:
            print("["+str(i)+"] "+Class.text.split("\n")[0])
            i = i + 1

        print("["+str(len(Classes))+"] "+"All Class")
        Class_chosen = int(input("Please Class's number: ").strip())

        if (Class_chosen >= 0 and Class_chosen <= len(Classes)):
            return i
        
# Going back on Driver n times
def goback(how_many_times):
    for i in range(0, how_many_times):
        Driver.back()
        time.sleep(10)
        
# Scrolling to body of the DOM
def scroll_down(n):
    try:
        for i in range(0, n):
            time.sleep(4)
            Driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            Button = Driver.find_element_by_css_selector("div.uArJ5e.UQuaGc.kCyAyd.nZ34k")
            Button.click()
        return False
    except:
        return True

# Scrolling to head of the DOM
def scroll_up():
    Driver.execute_script('window.scrollTo(0, document.head.scrollHeight);')

In [6]:
# Returning Assignment tab url
def assignment_tab():
    tabs = Driver.find_elements_by_css_selector("div.wZTANe")
    return (tabs[1].find_element_by_tag_name("a")).get_attribute("href")

# Going to inside of the assignment and extract assignment's information
def goto_assignment(assignment_index):
    Assignments = Driver.find_elements_by_css_selector('li[guidedhelpid="classworkTopicListGh"]')
    Assignments[assignment_index].click()
    time.sleep(2)
    
    data = Assignments[assignment_index].text.split("\n")
    date = Assignments[assignment_index].find_element_by_css_selector("div.wCDkmf.T3FW5d.YVvGBb.dDKhVc").text.split("Tenggat: ")
    assignment_id = Assignments[assignment_index].get_attribute("data-dom-id").split("|")
    try:
        Assignments[assignment_index].find_element_by_css_selector("a.IrxBzb.TpQm9d").click()

        return {
            "assignment_name" : data[0],
            "assignment_id" : assignment_id[0],
            "deadlines" : date[len(date)-1]
        }
    except:
        assignment_index = assignment_index + 1

# Inside of the assignment, scrap student's information
def scrap_student(assignment_index):
    students_list = []
    try:
        time.sleep(10)
        Students = Driver.find_elements_by_css_selector("tr.DC55n")
        for i in range(0, len(Students)):
            Student_button = Students[i].find_element_by_tag_name("a")
            Student_id = Students[i].get_attribute("data-student-id")
            Student_button.click()
            time.sleep(3)
            try:
                Driver.find_elements_by_css_selector('div.ZSrdFd.Y5vSD.x4nQ7.VBEdtc-Wvd9Cc.zZN2Lb-Wvd9Cc')[i].click()
                time.sleep(2)
                Information = Driver.find_element_by_css_selector("table.kbngx.IMvYId").find_elements_by_tag_name("td")
                
                students_list.append({
                    "student_name" : Information[2].text,
                    "student_id" : Student_id,
                    "status" : Information[0].text,
                    "collection_time" : Information[1].text
                })
                Driver.find_element_by_css_selector('div.uArJ5e.UQuaGc.kCyAyd.l3F1ye.ARrCac.HvOprf.M9Bg4d').click()
            except:
                Information = Driver.find_elements_by_css_selector("div.C2AFcd")[i].text.split("\n")
                
                students_list.append({
                    "student_name" : Information[0],
                    "student_id" : Student_id,
                    "status" : Information[1]
                })
        
        return students_list
    except:
        pass

In [7]:
# Going to class
class_index = 0 # search_class()
classes = Driver.find_elements_by_css_selector("div.Tc9hUd")
classes[class_index].click()
time.sleep(5)

In [8]:
# Scraping course
def scrap_course():
    # Going to Assignment tab
    assignment_tab_url = assignment_tab()
    Driver.get(assignment_tab_url)
    time.sleep(5)
    assignment_index = 0
    
    # Extracting Course's name
    course_name = Driver.find_element_by_id("UGb2Qe").text

    assignments_list = []
    # Going to each assignment and try to scrap it
    while (True):
        try:
            # Scraping assignment's information
            assignment_info = goto_assignment(assignment_index)
            # Scraping student's information corresponding to current assignment 
            student_assignment_info = scrap_student(assignment_index)
            
            assignments_list.append({
                "assignment_info" : assignment_info,
                "students_assignment_info" : student_assignment_info
            })
            Driver.get(assignment_tab_url)
            assignment_index = assignment_index + 1
        except:
            # If it's ending of the assignment tab, stop this while loop
            Ending = scroll_down(1)
            if(Ending):
                break

        time.sleep(10)
    
    # Returning course's information and it's assignments' information
    return {
        "course_name" : course_name,
        "assignment" : assignments_list
    }

In [11]:
result = scrap_course()

In [12]:
result

{'course_name': 'Basis Data 2A/D3',
 'assignment': [{'assignment_info': {'assignment_name': 'Perbaikan Nilai UTS',
    'assignment_id': '48542288855',
    'deadlines': '3 Jan 17.00'},
   'students_assignment_info': [{'student_name': 'ADHITYA FEBHIAKBAR -',
     'student_id': '44550928300',
     'status': 'Diserahkan',
     'collection_time': '3 Jan 16.24'},
    {'student_name': 'AJI MUHAMMAD ZAPAR -',
     'student_id': '44550140855',
     'status': 'Diserahkan',
     'collection_time': '3 Jan 13.17'},
    {'student_name': 'ARIS PURNOMO -',
     'student_id': '44551525394',
     'status': 'Diserahkan',
     'collection_time': '3 Jan 13.24'},
    {'student_name': 'ELZA ESTERINA -',
     'student_id': '44555621635',
     'status': 'Diserahkan',
     'collection_time': '3 Jan 16.30'},
    {'student_name': 'FAHMI WIDIANTO -',
     'student_id': '44553938654',
     'status': 'Diserahkan',
     'collection_time': '2 Jan 00.03'},
    {'student_name': 'GINA MELINIA -',
     'student_id': '4455

In [13]:
# Importing JSON
import json

In [14]:
# Create file and save result as a JSON file
file = open('Google Classroom_Instructor_Scraped by Ali Nurdin.json', 'w')
json.dump(result, file)
file.close()